`Dataframe` to `DocumentArray`...

In [38]:
from docarray import Document, DocumentArray
import sys

sys.path.append('C:/Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML')
from data_preprocess import data_selection
import metrics

train_df, test_df = data_selection('C:/Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML/data/utkface_debiased/')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (559904971.py, line 8)

Initializing `CLIP`...

In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [ ]:
import finetuner

finetuner.login()

`Race ` and ` Gender` Fine-tuning...

In [ ]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'] + ' gender and ' + doc.tags['race'] + ' race.' )
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

📄 Document: fee1dbdb709c89e0e0974edc6fc908e0
└── 💠 Chunks
    ├── 📄 Document: 736f3346fb2d969a46615700f90cafcc
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122135009… │
    │   │ tags      │ {'filename': '100_0_0_20170112213500903.jpg.chip.jpg', 'gender': │
    │   │           │ 'male', 'age': 100, 'filepath':                                  │
    │   │           │ 'C:/Users/lachy/Pictures/utkface/utkface/100_0_0_20170112213500… │
    │   │           │ 'race': 'white'}                                                 │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: 5a9048c818ce4b14130f83432b76ae01
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [ ]:
for doc in gender_pairs:
    doc.chunks[0].tags = None
gender_pairs[0]


📄 Document: d1aafaf562d5dde11bec9900150cf352
└── 💠 Chunks
    ├── 📄 Document: e177d4d5b1a155a63e8055e689bb4457
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122152403… │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: f140088be73e6a65dd0737fe403d5e9a
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [ ]:
len(gender_pairs)

18967

Training `CLIP` model...

In [ ]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32', # fine-tune CLIP
    train_data=gender_pairs,   
    learning_rate=1e-5,
    loss='CLIPLoss'
)

Pushing a DocumentArray to Hubble under the name finetuner-dastorage-default-stoic-newton-train ...


🔐 You are logged in to Jina AI as de.todito70y19 (username: TuKola). To log out, use jina auth logout.

Output()

In [ ]:
for entry in run.stream_logs():
    print(entry)

Output()

[08:10:23] INFO     Starting finetuner training run ...                                                  __main__.py:350
DEBUG    Found Jina AI Cloud authentication token                                             __main__.py:362
DEBUG    Running in online mode                                                               __main__.py:363
INFO     Reading config ...                                                                   __main__.py:370
DEBUG    Reading config from stream                                                           __main__.py:382
INFO     Parsing config ...                                                                   __main__.py:385
INFO     Config loaded 📜                                                                     __main__.py:389
INFO     Run name: stoic-newton                                                               __main__.py:391
INFO     Experiment name: default                                                             __main__.py:392


Saving the `artifact`

In [ ]:
artifact = run.save_artifact('/artifact/')

Output()

Saving `encoders`...

In [ ]:
clip_text_encoder = finetuner.get_model(artifact='/artifact/stoic-newton.zip', select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact='/artifact/stoic-newton.zip', select_model='clip-vision')

Saving `images embeddings`...

In [ ]:
for d in test_da:
    d.uri = d.tags['filepath']
images = finetuner.encode(model=clip_image_encoder, data=test_da)

Output()

Create `query texts` to evaluate the model...

In [ ]:
#Race query texts
race_labels = ['black', 'white', 'asian', 'indian', 'other']
race_tks = []
for label in race_labels:
    race_tks.append('A photo of a person of ' + label + ' race.')

#Sex query texts
sex_labels = ['male', 'female']
sex_tks = []
for label in sex_labels:
    sex_tks.append('A photo of a person of ' + label + ' gender.')




Saving `sex texts` embeddings...

In [ ]:
text_docs = DocumentArray()
for item in sex_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: 25e2af94cc92c39750b47825364c64ae
╭──────────────────┬───────────────────────────────────────────────────────────╮
│ Attribute        │ Value                                                     │
├──────────────────┼───────────────────────────────────────────────────────────┤
│ text             │ A photo of a person of male gender.                       │
╰──────────────────┴───────────────────────────────────────────────────────────╯

📄 Document: 74279661774e0357598b840f91f68795
╭─────────────────┬────────────────────────────────────────────────────────────╮
│ Attribute       │ Value                                                      │
├─────────────────┼────────────────────────────────────────────────────────────┤
│ text            │ A photo of a person of female gender.                      │
╰─────────────────┴────────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 2                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   2                False             │
│   id          ('str',)       2                False             │
│   text        ('str',)       2                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model for `sex predictions`...

In [ ]:
images.match(text_docs, metric='cosine', limit=1)
y_true = [image.tags['gender'] for image in images]
y_predict = [image.matches[0].text.split(' ')[-2] for image in images]
labels = ['male', 'female']

## Detecting bias in finetuned `CLIP ` (sex results)

`Disparate impact`

In [ ]:
metrics.disparate_impact(y_true, y_predict, labels)

DISPARATE IMPACT.
No disparate impact present


False

`Equalized odds`

In [ ]:
metrics.equalized_odds(y_true, y_predict, labels)

EQUALIZED ODDS
Equalized odds


True

## `Sex` Evaluation Metrics

`Macro accuracy`

In [ ]:
metrics.macro_accuracy(y_true, y_predict)

0.9597124603715524

`Macro F1`

In [ ]:
metrics.macro_f1(y_true, y_predict, labels)

0.9580240522177677

`Confusion matrix`

In [ ]:
metrics.confusion_matrix(y_true, y_predict, labels, plot=False)

array([[0.92251816, 0.07748184],
       [0.00309324, 0.99690676]])

## Detecting bias in finetuned `CLIP ` (race results)

Saving `race texts` embeddings...

In [ ]:
text_docs = DocumentArray()
for item in race_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: 553df6163cffb9447d966c3c0e00aadf
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of black race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

📄 Document: 303b80c76ac3459d3251e038c35aa885
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of white race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 5                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   5                False             │
│   id          ('str',)       5                False             │
│   text        ('str',)       5                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model for `race predictions`...

In [ ]:
images.match(text_docs, metric='cosine', limit=1)
y_true = [image.tags['race'] for image in images]
y_predict = [image.matches[0].text.split(' ')[-2] for image in images]
labels = ['white', 'black', 'asian', 'indian', 'other']


`Disparate impact`




In [ ]:
metrics.disparate_impact(y_true, y_predict, labels)

DISPARATE IMPACT.
Disparate impact present in  white/black
Value:0.3877758354374957
Disparate impact present in  white/asian
Value:0.2646690073029772
Disparate impact present in  white/indian
Value:0.31874743182723336
Disparate impact present in  white/other
Value:0.13593860561914675
Disparate impact present in  black/asian
Value:0.6825309447257663
Disparate impact present in  black/other
Value:0.35055976467893707
Disparate impact present in  asian/other
Value:0.5136173933033737
Disparate impact present in  indian/other
Value:0.42647749297891696


True

`Equalized odds`

In [ ]:
metrics.equalized_odds(y_true, y_predict, labels)

EQUALIZED ODDS
0.12237906649357189: Not equalized odds between white and black (tpr). Privileged group: asian
0.22110950982623045: Not equalized odds between white and asian (tpr). Privileged group: asian
0.18243987702295672: Not equalized odds between white and indian (tpr). Privileged group: asian


False

## `Race` Evaluation Metrics

`Macro accuracy`

In [ ]:
metrics.macro_accuracy(y_true, y_predict)

0.9083313291491779

`Macro F1`

In [ ]:
metrics.macro_f1(y_true, y_predict, labels)

0.8527411077502229

`Confusion matrix`

In [ ]:
metrics.confusion_matrix(y_true, y_predict, labels, plot = False)

array([[0.76724566, 0.0044665 , 0.00099256, 0.00794045, 0.21935484],
       [0.00220751, 0.88962472, 0.00110375, 0.00220751, 0.10485651],
       [0.        , 0.        , 0.98835517, 0.00582242, 0.00582242],
       [0.00377358, 0.00251572, 0.        , 0.94968553, 0.04402516],
       [0.00591716, 0.00887574, 0.00295858, 0.03550296, 0.94674556]])